In [2]:
%pip install scanpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 17.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=a058cfde0e769f2f2af67ea61270e87fd894f99c272738e84fe6f40c6237dde0
  Stored in directory: /Users/osolovei/Library/Caches/pip/wheels/23/da/7c/868424f4a5845ab58cd8686e0eb405e9e1e2d4152bf702c39d
Successfully built session-info
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required packages
import scanpy as sc
import pandas as pd
import numpy as np

# Load example brain dataset
adata = sc.datasets.pbmc68k_reduced()

# Basic preprocessing steps
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Normalize data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Convert to pandas DataFrame - removed .toarray() since X is already an array
expression_df = pd.DataFrame(adata.X,
                           columns=adata.var_names,
                           index=adata.obs_names)

# Add cell type labels as a column
expression_df['cell_type'] = adata.obs['louvain']

# Save basic info
print("Dataset dimensions:", expression_df.shape)
print("\nNumber of cell types:", len(expression_df['cell_type'].unique()))
print("\nCell type distribution:\n", expression_df['cell_type'].value_counts())

# Save to CSV
expression_df.to_csv('brain_sc_expression.csv', index=True)

# Optional: Save a smaller subset if the file is too large
# Select top 2000 most variable genes
gene_vars = expression_df.iloc[:, :-1].var()  # exclude cell_type column
top_genes = gene_vars.nlargest(2000).index

subset_df = expression_df[list(top_genes) + ['cell_type']]
subset_df.to_csv('brain_sc_expression_subset.csv', index=True)

/opt/miniconda3/envs/ds/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/opt/miniconda3/envs/ds/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:383: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


AttributeError: 'numpy.ndarray' object has no attribute 'toarray'